In [ ]:
"""
Explore speaker relationship with network analysis:

nodes: speakers
    - color: node['profession'] = 'entertainment','tech','education','author'...? TBD
    - size: degree centrality

edge: defined by overlap of words used


"""

In [2]:
import re, itertools
import networkx as nx

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations, count

## 1. Load data

In [97]:
speech_df = pd.read_pickle('../dump/speech_clean_lemma')
speaker_info = pd.read_pickle('../dump/speaker_info')

In [98]:
speech_df

,speaker,year,transcript,length
0,SIDDHARTHA MUKHERJEE,2018,i wish someone have tell me at my own commence...,14487
1,ABBY WAMBACK,2018,failure be not something to be ashamed of its ...,15866
2,JON B. FISHER,2018,thank you very much my father commencement spe...,8544
3,MINDY KALING,2018,good morning to the class of the faculty the p...,15391
4,JESMYN WARD,2018,persist be patient be well good morning it be ...,14063
...,...,...,...,...
436,CARRIE CHAPMAN,1936,i bring a message to sweet briar college and e...,22942
437,FRANKLIN D ROOSEVELT,1932,for me as for you this be a day of honorable a...,16017
438,OPRAH WINFREY,1918,thank you wallis annenberg and a special thank...,15301
439,RALPH WALDO,1838,in this refulgent summer it have be a luxury t...,40403


In [99]:
speaker_info

,speaker,profession,born,age,origin
0,SIDDHARTHA MUKHERJEE,American-Indian physician,1970,50,New Delhi India
1,JESMYN WARD,American novelist,1977,43,De Lisle MS
2,DAVID SEDARIS,American humorist,1956,64,Johnson City NY
3,NIGHT SHYAMALAN,American-Indian filmmaker,1970,50,Mahé India
4,TERRY TEACHOUT,American author,1956,65,Cape Girardeau MO
...,...,...,...,...,...
436,WILLIAM ALLEN,Attorney,1704,NaN,Philadelphia PA
437,FRANKLIN D ROOSEVELT,32nd U.S. President,1882,NaN,Hyde Park NY
438,OPRAH WINFREY,American host,1954,67,Kosciusko MS
439,RALPH WALDO,American essayist,1803,NaN,Boston MA


## 2. Fill missing values

### Profession

In [123]:
update_profession = [
    'writer', 'actor', 'Educator', 'Professor', 'CEO of Right to Start', 
    'CEO of Microsoft', 'Professor', 'Poet', 'spiritual leader', 'Attorney',
    'Finance', 'Educator', 'Biologist', 'Tech (Cisco)', 'Environmental diplomat',
    'Pulitzer Prize winner author', 'Professor', 'Professor', 'Student', 'Tech (Instagram)',
    'spiritual (pastor)', 'Educator', 'Finance (investor)', 'Finance (investor)', 'Politician',
    'student', 'Culture (911 museum)', 'spiritual (pastor)', 'writer', 'writer',
    '??', 'Tech (Instagram)', 'Tech, educator', 'Tech, educator','Science (NASA)',
    'CEO (NanoHorizons)','writer', 'writer', 'CEO (Coca-cola)', 'CEO (Coca-cola)', 
    'student', 'Judge'
]

In [125]:
len(update_profession)

42

In [132]:
test_df['profession_update'] = update_profession

<ipython-input-132-3486a7b2cddb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['profession_update'] = update_profession


In [133]:
test_df

,speaker,profession,born,age,origin,profession_update
10,KELLY MCMASTERS,NaN,nan,NaN,Books: Welcome to Shirley: A Memoir from an At...,writer
29,JAMES RYAN,NaN,nan,NaN,NaN,actor
39,DONOVAN LIVINGSTON,NaN,nan,NaN,Books: Lift Off: From the Classroom to the Stars,Educator
86,EMIR KAMENICA,NaN,nan,NaN,h-index: 25,Professor
87,VICTOR HWANG,NaN,1971,49,Born: May 27 1971 (age 49 years),CEO of Right to Start
90,BILL & MELINDA GATES,NaN,nan,NaN,Headquarters: Seattle WA,CEO of Microsoft
111,STEPHEN R KELLERT,NaN,nan,NaN,Edited works: The Biophilia Hypothesis MORE,Professor
120,BILL COLLINS,NaN,nan,NaN,NaN,Poet
126,DALAI LAMA,NaN,1935,85,Taktser China,spiritual leader
129,JOHN JACOB SCHERER,NaN,nan,NaN,NaN,Attorney


In [100]:
update = ['writer']*42


In [118]:
test_df.loc[test_df.profession.isna(),'profession']=

/Users/katiehuang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


ValueError: cannot copy sequence with size 42 to array axis with dimension 39

In [117]:
test_df

,speaker,profession,born,age,origin
10,KELLY MCMASTERS,writer,nan,NaN,Books: Welcome to Shirley: A Memoir from an At...
29,JAMES RYAN,writer,nan,NaN,NaN
39,DONOVAN LIVINGSTON,writer,nan,NaN,Books: Lift Off: From the Classroom to the Stars
86,EMIR KAMENICA,NaN,nan,NaN,h-index: 25
87,VICTOR HWANG,NaN,1971,49,Born: May 27 1971 (age 49 years)
90,BILL & MELINDA GATES,NaN,nan,NaN,Headquarters: Seattle WA
111,STEPHEN R KELLERT,NaN,nan,NaN,Edited works: The Biophilia Hypothesis MORE
120,BILL COLLINS,NaN,nan,NaN,NaN
126,DALAI LAMA,NaN,1935,85,Taktser China
129,JOHN JACOB SCHERER,NaN,nan,NaN,NaN


In [101]:
pd.Series(update)

0     writer
1     writer
2     writer
3     writer
4     writer
5     writer
6     writer
7     writer
8     writer
9     writer
10    writer
11    writer
12    writer
13    writer
14    writer
15    writer
16    writer
17    writer
18    writer
19    writer
20    writer
21    writer
22    writer
23    writer
24    writer
25    writer
26    writer
27    writer
28    writer
29    writer
30    writer
31    writer
32    writer
33    writer
34    writer
35    writer
36    writer
37    writer
38    writer
39    writer
40    writer
41    writer
dtype: object

In [94]:
speaker_info[speaker_info.profession.isna()]['profession']='writer'

<ipython-input-94-c635a0ee1318>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speaker_info[speaker_info.profession.isna()]['profession']='writer'


In [95]:
[]

,speaker,profession,born,age,origin
0,SIDDHARTHA MUKHERJEE,American-Indian physician,1970,50,New Delhi India
1,JESMYN WARD,American novelist,1977,43,De Lisle MS
2,DAVID SEDARIS,American humorist,1956,64,Johnson City NY
3,NIGHT SHYAMALAN,American-Indian filmmaker,1970,50,Mahé India
4,TERRY TEACHOUT,American author,1956,65,Cape Girardeau MO
...,...,...,...,...,...
436,WILLIAM ALLEN,Attorney,1704,NaN,Philadelphia PA
437,FRANKLIN D ROOSEVELT,32nd U.S. President,1882,NaN,Hyde Park NY
438,OPRAH WINFREY,American host,1954,67,Kosciusko MS
439,RALPH WALDO,American essayist,1803,NaN,Boston MA


In [119]:
test_df = speaker_info[speaker_info['profession'].isna()]
test_df.shape

(42, 5)

In [84]:
test_list = ['hi']*42

In [88]:
test_df['profession'] = pd.Series(test_list)

<ipython-input-88-47c53f0bb3c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['profession'] = pd.Series(test_list)


In [89]:
test_df.iloc[10]['profession'] = "writer"

,speaker,profession,born,age,origin,profession_update
10,KELLY MCMASTERS,hi,NaN,NaN,Books: Welcome to Shirley: A Memoir from an At...,hi
29,JAMES RYAN,hi,NaN,NaN,NaN,hi
39,DONOVAN LIVINGSTON,hi,NaN,NaN,Books: Lift Off: From the Classroom to the Stars,hi
86,EMIR KAMENICA,NaN,NaN,NaN,h-index: 25,NaN
87,VICTOR HWANG,NaN,1971,49,Born: May 27 1971 (age 49 years),NaN
90,BILL & MELINDA GATES,NaN,NaN,NaN,Headquarters: Seattle WA,NaN
111,STEPHEN R KELLERT,NaN,NaN,NaN,Edited works: The Biophilia Hypothesis MORE,NaN
120,BILL COLLINS,NaN,NaN,NaN,NaN,NaN
126,DALAI LAMA,NaN,1935,85,Taktser China,NaN
129,JOHN JACOB SCHERER,NaN,NaN,NaN,NaN,NaN


### Year born

In [40]:
speaker_info['born'] = speaker_info['born'].replace("nan",np.nan)
speaker_info[speaker_info['born'].isna()]

,speaker,profession,born,age,origin
10,KELLY MCMASTERS,NaN,NaN,NaN,Books: Welcome to Shirley: A Memoir from an At...
28,KENNETH KOSIK,Neurologist,NaN,NaN,h-index: 107
29,JAMES RYAN,NaN,NaN,NaN,NaN
39,DONOVAN LIVINGSTON,NaN,NaN,NaN,Books: Lift Off: From the Classroom to the Stars
62,CODY KEENAN,Former White House Director of Speechwriting,NaN,NaN,Born: Chicago IL
82,TORY BURCH,Fashion company,NaN,NaN,Founder: Tory Burch
86,EMIR KAMENICA,NaN,NaN,NaN,h-index: 25
90,BILL & MELINDA GATES,NaN,NaN,NaN,Headquarters: Seattle WA
111,STEPHEN R KELLERT,NaN,NaN,NaN,Edited works: The Biophilia Hypothesis MORE
120,BILL COLLINS,NaN,NaN,NaN,NaN


In [41]:
speaker_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441 entries, 0 to 440
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   speaker     441 non-null    object
 1   profession  399 non-null    object
 2   born        397 non-null    object
 3   age         331 non-null    object
 4   origin      418 non-null    object
dtypes: object(5)
memory usage: 17.4+ KB


In [49]:
speaker_info[speaker_info['born'].isnull()]

,speaker,profession,born,age,origin
10,KELLY MCMASTERS,NaN,NaN,NaN,Books: Welcome to Shirley: A Memoir from an At...
28,KENNETH KOSIK,Neurologist,NaN,NaN,h-index: 107
29,JAMES RYAN,NaN,NaN,NaN,NaN
39,DONOVAN LIVINGSTON,NaN,NaN,NaN,Books: Lift Off: From the Classroom to the Stars
62,CODY KEENAN,Former White House Director of Speechwriting,NaN,NaN,Born: Chicago IL
82,TORY BURCH,Fashion company,NaN,NaN,Founder: Tory Burch
86,EMIR KAMENICA,NaN,NaN,NaN,h-index: 25
90,BILL & MELINDA GATES,NaN,NaN,NaN,Headquarters: Seattle WA
111,STEPHEN R KELLERT,NaN,NaN,NaN,Edited works: The Biophilia Hypothesis MORE
120,BILL COLLINS,NaN,NaN,NaN,NaN


In [52]:
speaker_info['born'].fillna(0).unique()

array(['1970', '1977', '1956', '1980', '1979', '1972', '1973', '1953', 0,
       '1945', '1962', '1967', '1984', '1969', '1954', '1978', '1948',
       '1960', 'Born:', '1947', '1988', '1946', '1964', '1952', '1936',
       '1949', '1958', '1959', '1961', '1974', '1950', '1955', '1943',
       '1971', '1934', '1981', '1982', '1965', '1976', '1951', '1983',
       '1957', '1991', '1975', '1942', '1944', '1935', '1963', '1985',
       '1987', '1937', '1933', '1966', '1931', '1929', '1968', '1938',
       '1940', '1941', '1803', '1921', '1928', '1927', '1924', '1923',
       '1920', '1922', '1932', '1913', '1919', '1925', '1918', '1930',
       '1939', '1907', '1910', '1904', '1926', '1908', '1917', '1911',
       '1890', '1880', '1888', '1859', '1704', '1882'], dtype=object)

In [54]:
wrong_born = speaker_info[speaker_info['born'] == "Born:"]

In [56]:
def correct_born(row):
    row['born'] = 2021 - row['age']

In [57]:
wrong_born.apply(correct_born)

KeyError: 'age'

In [43]:
speaker_info['age_now'] = 2020 - speaker_info['born'].fillna(0).astype(int)

ValueError: invalid literal for int() with base 10: 'Born:'

## 3. Network analysis